<a href="https://colab.research.google.com/github/Pavitbagga/FURI-2024/blob/main/Project1CSE575.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports and read from tensorflow.

In [ ]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
#!pip install tensorflow
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import pandas as pd

# Load the MNIST dataset
(ds_train, ds_test), _ = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,  # Load the data in tuple format (input, label)
    with_info=True,
    batch_size=-1  # Load the full dataset in one batch
)

train_images, train_labels = tfds.as_numpy(ds_train)

# Function to filter images and their labels based on the label value
def filter_images_and_labels(images, labels, label_value):
    mask = labels == label_value
    return images[mask], labels[mask]

# Filter images and labels for class 7 and 8
images_7, labels_7 = filter_images_and_labels(train_images, train_labels, 7)
images_8, labels_8 = filter_images_and_labels(train_images, train_labels, 8)

print(images_7.shape)
print(images_8.shape)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.
(6265, 28, 28, 1)
(5851, 28, 28, 1)


Calculate Features

In [ ]:
images_7_flat = images_7.reshape(images_7.shape[0], -1)
images_8_flat = images_8.reshape(images_8.shape[0], -1)

# Function to calculate the mean and standard deviation for images
def calculate_features(images):
    # Calculate the mean for each image
    means = np.mean(images, axis=1)
    # Calculate the standard deviation for each image
    std_devs = np.std(images, axis=1)
    return means, std_devs

# Calculate features for images labeled as 7
mean_7, std_dev_7 = calculate_features(images_7_flat)
# Calculate features for images labeled as 8
mean_8, std_dev_8 = calculate_features(images_8_flat)

# Bayes Classifier

Recall Bayes Theorum P(A|B) = P(B|A)P(A)/P(B)

> Add blockquote



In [ ]:
size_total = images_7.shape[0]+images_8.shape[0]
prior_7 = images_7.shape[0]/size_total
prior_8 = images_8.shape[0]/size_total
print(prior_7)
print(prior_8)

0.5170848464839881
0.4829151535160119


Parameters

In [ ]:
params_7 = {
    'mean_mean': np.mean(mean_7), 'var_mean': np.var(mean_7),
    'mean_std': np.mean(std_dev_7), 'var_std': np.var(std_dev_7)
}
params_8 = {
    'mean_mean': np.mean(mean_8), 'var_mean': np.var(mean_8),
    'mean_std': np.mean(std_dev_8), 'var_std': np.var(std_dev_8)
}

Gaussian and Classification

In [ ]:
def gaussian_pdf(x, mean, var):
    """ Probability density function for a Gaussian. """
    return (1 / np.sqrt(2 * np.pi * var)) * np.exp(-((x - mean) ** 2) / (2 * var))

def classify(image, params_7, params_8, prior_7, prior_8):
    mean, std_dev = np.mean(image), np.std(image)

    # Calculate log probabilities for class 7
    log_likelihood_7 = np.log(gaussian_pdf(mean, params_7['mean_mean'], params_7['var_mean']))
    log_likelihood_7 += np.log(gaussian_pdf(std_dev, params_7['mean_std'], params_7['var_std']))
    log_prob_7 = log_likelihood_7 + np.log(prior_7)

    # Calculate log probabilities for class 8
    log_likelihood_8 = np.log(gaussian_pdf(mean, params_8['mean_mean'], params_8['var_mean']))
    log_likelihood_8 += np.log(gaussian_pdf(std_dev, params_8['mean_std'], params_8['var_std']))
    log_prob_8 = log_likelihood_8 + np.log(prior_8)

    # Predict by comparing log probabilities
    if log_prob_7 > log_prob_8:
        return 7
    else:
        return 8

# Test

In [ ]:
test_images, test_labels = tfds.as_numpy(ds_test)
test_images_7, test_labels_7 = filter_images_and_labels(test_images, test_labels, 7)
test_images_8, test_labels_8 = filter_images_and_labels(test_images, test_labels, 8)
print("Test Images for class 7:", test_images_7.shape)
print("Test Images for class 8:", test_images_8.shape)

combined_test_images = np.concatenate((test_images_7, test_images_8), axis=0)
combined_test_labels = np.concatenate((test_labels_7, test_labels_8), axis=0)


Test Images for class 7: (1028, 28, 28, 1)
Test Images for class 8: (974, 28, 28, 1)


Classify for all

In [ ]:
predicted_labels_all = np.array([classify(img, params_7, params_8, prior_7, prior_8) for img in combined_test_images])


In [ ]:
true_labels = combined_test_labels
# Calculate overall accuracy
accuracy_all = np.mean(predicted_labels_all == true_labels)
print(f"Overall Accuracy of the classifier on the combined test set: {accuracy_all * 100:.2f}%")

Overall Accuracy of the classifier on the combined test set: 69.53%


For 7's

In [ ]:
indices_7 = (true_labels == 7)
correct_predictions_7 = predicted_labels_all[indices_7] == true_labels[indices_7]
accuracy_7 = np.mean(correct_predictions_7)
print(f"Accuracy for class 7: {accuracy_7 * 100:.2f}%")


Accuracy for class 7: 75.97%


For 8's

In [ ]:
indices_8 = (true_labels == 8)
correct_predictions_8 = predicted_labels_all[indices_8] == true_labels[indices_8]
accuracy_8 = np.mean(correct_predictions_8)
print(f"Accuracy for class 8: {accuracy_8 * 100:.2f}%")

Accuracy for class 8: 62.73%
